<a href="https://colab.research.google.com/github/sivaratrisrinivas/ttt-playground/blob/main/notebooks/05_ttt_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 3: TTT-Linear Layer Tests

Tests for each step in Phase 3 of plan.md

In [ ]:
# Setup: Clone repo and install dependencies
import sys
import os
from pathlib import Path

if 'ttt-playground' not in os.getcwd():
    !git clone https://github.com/sivaratrisrinivas/ttt-playground.git
    %cd ttt-playground

if 'content' in os.getcwd() and 'ttt-playground' in os.getcwd():
    os.chdir('/content/ttt-playground')
sys.path.insert(0, str(Path.cwd()))

!pip install -q -r requirements.txt
print(f"✓ Working directory: {os.getcwd()}")

## Step 3.1: Import models package

In [ ]:
from src.models import *
print("✓ Step 3.1: from src.models import * succeeds")

## Step 3.2: TTTLinear.__init__

In [ ]:
from src.models.ttt_linear import TTTLinear

layer = TTTLinear(768, 2048, 768)
print(f"✓ TTTLinear instantiated")
print(f"  W_h.shape: {layer.W_h.shape}")
assert layer.W_h.shape == (2048, 768), f"Expected (2048, 768), got {layer.W_h.shape}"
print("✓ Step 3.2: W_h.shape == (2048, 768) verified")

## Step 3.3: TTTLinear.forward (inference mode)

In [ ]:
import torch

# Test forward pass - inference mode
x = torch.randn(1, 128, 768)
y = layer(x, learning=False)
print(f"  Input shape: {x.shape}")
print(f"  Output shape: {y.shape}")
assert y.shape == (1, 128, 768), f"Expected (1, 128, 768), got {y.shape}"
print("✓ Step 3.3: Output shape [1, 128, 768] verified")

## Step 3.4: Initial weights stored for reset

In [ ]:
# Verify _W_h_initial exists and matches W_h
assert hasattr(layer, '_W_h_initial'), "Missing _W_h_initial attribute"
assert torch.allclose(layer.W_h, layer._W_h_initial), "W_h should match _W_h_initial after init"
print("✓ Step 3.4: _W_h_initial stored and matches W_h")

## Step 3.5: TTTLinear.forward (learning mode)

In [ ]:
# Create fresh layer for learning test
layer = TTTLinear(768, 2048, 768)
w_before = layer.W_h.clone()

x = torch.randn(1, 128, 768)
y = layer(x, learning=True)

assert not torch.allclose(layer.W_h, w_before), "W_h should change after learning=True"
print("✓ Step 3.5: W_h differs from initial after learning=True")

## Step 3.6: reset_weights()

In [ ]:
# Reset and verify
layer.reset_weights()
assert torch.allclose(layer.W_h, layer._W_h_initial), "W_h should match _W_h_initial after reset"
print("✓ Step 3.6: reset_weights() restores initial W_h")

## Step 3.7: get_weight_delta()

In [ ]:
# Learn then check delta
layer = TTTLinear(768, 2048, 768)
x = torch.randn(1, 128, 768)
layer(x, learning=True)

delta = layer.get_weight_delta()
print(f"  Weight delta: {delta}")
assert delta > 0, "Weight delta should be > 0 after learning"
print("✓ Step 3.7: get_weight_delta() > 0 after learning")

## Step 3.8: Gradient flow

In [ ]:
# Test gradient flow through layer
layer = TTTLinear(768, 2048, 768)
x = torch.randn(1, 128, 768, requires_grad=True)
y = layer(x, learning=False)
loss = y.sum()
loss.backward()

assert x.grad is not None, "Input gradient should not be None"
print("✓ Step 3.8: Gradient flows through layer (input.grad is not None)")

print("\n" + "="*50)
print("✓ ALL PHASE 3 TESTS PASSED!")
print("="*50)